# Common

In [3]:
# print_function for compatibility with Python 3
from __future__ import print_function
print('print function is ready to serve')

# Matplotlib for visualization
from matplotlib import pyplot as plt

# display plots in the notebook
%matplotlib inline

print function is ready to serve


In [4]:
# NumPy for numerical computing
import numpy as np
np.random.seed(123)
import random
random.seed(123)

# Pandas for DataFrames
import pandas as pd
pd.set_option('display.max_column', 100)

from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

import os
import gc
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean

Using TensorFlow backend.


In [5]:
import sys
sys.path.append('./utils')

from data import Data
from models import Models

In [6]:
PLANET_KAGGLE_ROOT = '/data/planet-data/'
if not os.path.exists(PLANET_KAGGLE_ROOT):
    PLANET_KAGGLE_ROOT = '/Users/jiayou/Documents/Kaggle Data/Amazon'

N_TAGS = 17
N_TRAIN = 40479
# N_TRAIN = 10
N_TEST_T = 40669
N_TEST_F = 20522
N_TEST = N_TEST_T + N_TEST_F

In [5]:
# img = load_train_image(1)
# plt.imshow(img)
# img.shape

In [6]:
train_labels = pd.read_csv(os.path.join(PLANET_KAGGLE_ROOT, 'train_v2.csv'))
train_labels.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


# Training

In [9]:
def train(val=0, toy=None, d=None):
    print('')
    print('Training with val = {}'.format(val))
    print('')
    
    if d is None:
        d = Data(tif=False, toy=toy)

    m = Models.new_resnet50(input_shape=(256,256,3))

    if toy is None:
        h = m.fit_generator(
            d.gen_train(32, val=val), steps_per_epoch=1000,
            epochs=30, initial_epoch=0,
            validation_data=d.gen_val(100, val=val), validation_steps=80,
            callbacks=[
                ModelCheckpoint('weights-v9-f{}.hdf5'.format(val), save_best_only=True, verbose=1),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7, verbose=1)],
            max_q_size=10)
    else:
        h = m.fit_generator(
            d.gen_train(1), steps_per_epoch=8,
            epochs=30, initial_epoch=0,
            validation_data=d.gen_val(2), validation_steps=1,
            callbacks=[
                ModelCheckpoint('weights-v9-f{}.hdf5'.format(val), save_best_only=True, verbose=1),
                ReduceLROnPlateau(monitor='loss', factor=0.2, patience=1, min_lr=1e-7, verbose=1)],
            max_q_size=10)
        
    return h

In [10]:
toy = None
results = []
d = Data(tif=False, toy=toy)
for i in [1,2,3,4]:
    r = train(val=i, d=d, toy=toy)
    results.append(r)
    gc.collect()

Loading data...
Getting 2 training images...
Got 1 images
Done
Loaded fold 0.
Getting 2 training images...
Got 1 images
Done
Loaded fold 1.
Getting 2 training images...
Got 1 images
Done
Loaded fold 2.
Getting 2 training images...
Got 1 images
Done
Loaded fold 3.
Getting 2 training images...
Got 1 images
Done
Loaded fold 4.
Loading done

Training with val = 1

Epoch 1/30
8/8 [==============================] - 59s - loss: 0.6132 - amazon_score: 0.6708 - acc: 0.8382 - val_loss: 0.5310 - val_amazon_score: 0.4000 - val_acc: 0.8824
Epoch 2/30
8/8 [==============================] - 17s - loss: 0.2146 - amazon_score: 0.8106 - acc: 0.9191 - val_loss: 0.4277 - val_amazon_score: 0.4808 - val_acc: 0.7941
Epoch 3/30
8/8 [==============================] - 19s - loss: 0.3393 - amazon_score: 0.7509 - acc: 0.8456 - val_loss: 0.3886 - val_amazon_score: 0.5263 - val_acc: 0.8824
Epoch 4/30
7/8 [=========================>....] - ETA: 1s - loss: 0.3457 - amazon_score: 0.6873 - acc: 0.8655 

KeyboardInterrupt: 

In [2]:
# m, h = train(10)